In [1]:
from gurobipy import *
import csv
import math
import re
import pandas as pd

In [2]:
def data_load(fileName):
    dataFrame = []
    df_temp = pd.DataFrame

    with open(fileName, "r") as file:
        for line in file:
            if re.match("#",line) != None:
                #
                if df_temp.empty == False:
                    dataFrame.append(df_temp)
                    print(df_temp)
                    #  del df_temp
                str = line.split(": ")[1].replace(',\n', '')
                df_temp = pd.DataFrame(columns = str.split(", "))
            else:
                line = line.replace(',\n', '')
                val = line.split(", ")
                df_temp_length = len(df_temp)
                df_temp.loc[df_temp_length] = val
        dataFrame.append(df_temp)

    for i in range(len(dataFrame)):
        dataFrame[i] = dataFrame[i].set_index('loc_id', drop=True)
        # print(dataFrame[i])

    df_hospitals = dataFrame[0]
    df_existing_hospitals = dataFrame[1]
    df_cities = dataFrame[2]
    df_cities_minimum = dataFrame[3]

    return df_hospitals, df_existing_hospitals, df_cities, df_cities_minimum


In [3]:
df_hospitals, df_existing_hospitals, df_cities, df_cities_minimum = data_load('data1.csv')

  loc_id x_coord y_coord costk1 costk2 costk3 capk1 capk2 capk3
0     h1       0      10    100    200    300     1     2     3
1     h2      10      10    -10      0    150     1     3     4
2     h3       0      40     50    250    300     1     2     3
3     h4      40      10    180    200    480     2     2     3
4     h5      20      20    200    300    400     1     2     3
  loc_id closing_income
0     h2            100
  loc_id x_coord y_coord
0     c1       5       5
1     c2      25      10
2     c3      10      15
3     c4       5      20
4     c5       5      25
5     c6      40      20
6     c7      30      30
7     c8      65      10


In [4]:
df_hospitals

,x_coord,y_coord,costk1,costk2,costk3,capk1,capk2,capk3
loc_id,,,,,,,,
h1,0,10,100,200,300,1,2,3
h2,10,10,-10,0,150,1,3,4
h3,0,40,50,250,300,1,2,3
h4,40,10,180,200,480,2,2,3
h5,20,20,200,300,400,1,2,3


In [5]:
df_existing_hospitals

,closing_income
loc_id,
h2,100


In [6]:
df_cities

,x_coord,y_coord
loc_id,,
c1,5,5
c2,25,10
c3,10,15
c4,5,20
c5,5,25
c6,40,20
c7,30,30
c8,65,10


In [7]:
df_cities_minimum

""
loc_id
c1
c3
c6


In [8]:
def euclidean_distance(x1,y1,x2,y2):
    return math.sqrt((int(x1) - int(x2))**2 + (int(y1) - int(y2))**2)

In [9]:
def check_dist(df_hospitals, df_cities):
    id_pairs = []
    hospitals = df_hospitals.index.to_list() # J
    cities = df_cities.index.to_list() # I
    for i in cities:
        for j in hospitals:
            x_1, y_1 = df_hospitals.loc[j]['x_coord'], df_hospitals.loc[j]['y_coord']
            x_2, y_2 = df_cities.loc[i]['x_coord'], df_cities.loc[i]['y_coord']
            dist = euclidean_distance(x_1, y_1, x_2, y_2)
            if dist <= 30:
                id_pairs.append((j , i))
    return id_pairs


In [10]:
id_pairs = check_dist(df_hospitals, df_cities)

In [17]:
def solve(full_path_instance):
    df_hospitals, df_existing_hospitals, df_cities, df_cities_minimum = data_load(full_path_instance)
    
    #check reasonable distance
    id_pairs = check_dist(df_hospitals, df_cities)

    df_hospitals_cost = df_hospitals[['costk1', 'costk2', 'costk3']]
    df_hospitals_cap = df_hospitals[['capk1', 'capk2', 'capk3']]

    hospitals = df_hospitals.index.to_list() # J
    cities = df_cities.index.to_list() # I

    existing_hospitals = df_existing_hospitals.index.to_list() # J_2

    #type >= 2 cities
    cities_minimum = df_cities_minimum.index.to_list() # I_2

    types_hospitals = range(3)

    ############################################################################################
    model = Model("Hospitalnetwork")

    model.modelSense = GRB.MINIMIZE

    # whether hospital j for city i is used (value = 1) or not (value = 0).
    x = {}
    # for j in hospitals:
    #     for i in cities:
    #         x[j, i] = model.addVar(name="x_%s_%s" % (j, i), vtype=GRB.BINARY)
    for (j,i) in id_pairs:
        x[j, i] = model.addVar(name="x_%s_%s" % (j, i), vtype=GRB.BINARY)


    # which type k of hospital j in {1,2,3} is used (value = 1) or not (value = 0).
    y = {}
    # for j , _ in id_pairs:
    for j in hospitals:
        for k in range(3):
            y[j, k] = model.addVar(name="t_%s_%s" % (j, k), vtype=GRB.BINARY)

    
    # Maximum Cost C
    # C_max = model.addVar(vtype=GRB.CONTINUOUS)

    # model.setObjective(
    #     , GRB.MINIMIZE
    # )


    ############################################################################################
    print(id_pairs)
    #each city only have exactly one hospital
    # for j in hospitals:
    # for i in cities:
            # if (j , i) in id_pairs:
    for _, i in id_pairs:
        model.addConstr(quicksum(x[j,i] for j, city in id_pairs if city == i))

    # for j in hospitals:
    #     for i in cities:

    #build realtion between x and y
    for (j,i) in id_pairs:
            model.addConstr(x[j,i] <= quicksum(y[j,k] for k in types_hospitals) )

    #if use hospital j
    # for j , _ in id_pairs:
    for j in hospitals:
        model.addConstr( quicksum(y[j,k] for k in types_hospitals) <= 1)

    #type 1, 2: city access to a hospital within 20 km, type 3: within 30 km
    for (j,i) in id_pairs:
        model.addConstr(euclidean_distance(df_hospitals.loc[j]['x_coord'], df_hospitals.loc[j]['y_coord'],
                                            df_cities.loc[i]['x_coord'], df_cities.loc[i]['y_coord']) *
                        x[j,i] * y[j,0] <= 20)
        model.addConstr(euclidean_distance(df_hospitals.loc[j]['x_coord'], df_hospitals.loc[j]['y_coord'],
                                            df_cities.loc[i]['x_coord'], df_cities.loc[i]['y_coord']) *
                        x[j,i] * y[j,1] <= 20)
        model.addConstr(euclidean_distance(df_hospitals.loc[j]['x_coord'], df_hospitals.loc[j]['y_coord'],
                                            df_cities.loc[i]['x_coord'], df_cities.loc[i]['y_coord']) *
                        x[j,i] * y[j,2] <= 30)

    #some cities type >= 2:
    for i in cities_minimum:
        model.addConstr(quicksum(x[j,i] * quicksum(y[j,k] for k in [1,2]) for j ,city in id_pairs if city == i) == 1)

    # for i in cities_minimum:
    #     model.addConstr(quicksum(x[j,i] * y[j,0] for j in hospitals) == 0)

    # for j in hospitals:
    #     for k in types_hospitals:

    #hospital to city capacity
    for j in hospitals:
        for i in cities:
            if (j,i) in id_pairs:
                model.addConstr(quicksum(x[j,i] * y[j,k] for (j,i) in id_pairs for k in types_hospitals) <= int(df_hospitals_cap.loc[j][k]))

    #obj
    # s = 0
    # for j, i in id_pairs:
    #     for k in types_hospitals:
    #     s += x[j,i] * y[j,k] * int(df_hospitals_cost.loc[j][k])
    
    # t = 0
    # for j in existing_hospitals:
    #     temp = 0
    #     for k in types_hospitals:
    #         temp += y[j ,k]
    #     t += (1 - temp) * df_existing_hospitals.loc[j]['closing_income']
    
    # ans = s - t


    model.setObjective(
    quicksum(
        y[j,k] * int(df_hospitals_cost.loc[j][k]) 
        for j, _ in id_pairs 
        for k in types_hospitals
        )
    -
    quicksum( 
        ( 
            quicksum( (1 - y[j,k]) for k in types_hospitals)) * df_existing_hospitals.loc[j]['closing_income']  
            for j in existing_hospitals )
    )
            
    # model.addConstr(quicksum(x[j,i] * y[j,k] * int(df_hospitals_cost.loc[j][k]) for i in cities for j in hospitals for k in types_hospitals) -
    #                 quicksum(  (1 - quicksum(y[j,k] for k in types_hospitals) * int(df_existing_hospitals.loc[j]['closing_income']) for j in existing_hospitals) <= C_max)

    ############################################################################################
    model.update()
    model.optimize()

    def printSolution():
        if model.status == GRB.OPTIMAL:
            print('\n objective: %g\n' % model.ObjVal)
            print("Selected following matching:")
            for j in hospitals:
                for i in cities:
                    for k in types_hospitals:
                        if x[j,i].x == 1 and y[j,k].x == 1:
                            print((j,i,k+1,euclidean_distance(df_hospitals.loc[j]['x_coord'], df_hospitals.loc[j]['y_coord'],
                                               df_cities.loc[i]['x_coord'], df_cities.loc[i]['y_coord'])))
        else:
            print("No solution!")

    printSolution()

    return model

In [18]:
solve('data1.csv')

  loc_id x_coord y_coord costk1 costk2 costk3 capk1 capk2 capk3
0     h1       0      10    100    200    300     1     2     3
1     h2      10      10    -10      0    150     1     3     4
2     h3       0      40     50    250    300     1     2     3
3     h4      40      10    180    200    480     2     2     3
4     h5      20      20    200    300    400     1     2     3
  loc_id closing_income
0     h2            100
  loc_id x_coord y_coord
0     c1       5       5
1     c2      25      10
2     c3      10      15
3     c4       5      20
4     c5       5      25
5     c6      40      20
6     c7      30      30
7     c8      65      10
[('h1', 'c1'), ('h2', 'c1'), ('h5', 'c1'), ('h1', 'c2'), ('h2', 'c2'), ('h4', 'c2'), ('h5', 'c2'), ('h1', 'c3'), ('h2', 'c3'), ('h3', 'c3'), ('h5', 'c3'), ('h1', 'c4'), ('h2', 'c4'), ('h3', 'c4'), ('h5', 'c4'), ('h1', 'c5'), ('h2', 'c5'), ('h3', 'c5'), ('h5', 'c5'), ('h4', 'c6'), ('h5', 'c6'), ('h2', 'c7'), ('h4', 'c7'), ('h5', 'c7'), ('h4',

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'